In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_validate, GridSearchCV

In [45]:
df = pd.read_csv('demo_sales.csv')
df.tail()

,Date,Units,Sales
1088,27/12/2022,7114,22551.38
1089,28/12/2022,7759,24596.03
1090,29/12/2022,6457,20468.69
1091,30/12/2022,7284,23090.28
1092,31/12/2022,0,0.00


### Create features

In [ ]:
target_map = data['sales'].to_dict()

data['lag1'] = (data['date'] - pd.Timedelta('364 days')).map(target_map)
data['lag2'] = (data['date'] - pd.Timedelta('728 days')).map(target_map)
data['lag3'] = (data['date'] - pd.Timedelta('1092 days')).map(target_map)

In [49]:
#df['Date'] = df['Date'].str.replace('/', '-').head()
df['Date'] = pd.to_datetime(df['Date'], 
#               errors='ignore', 
                format='%d/%m/%Y',
                dayfirst=True)
df.tail()

,Date,Units,Sales
1088,2022-12-27,7114,22551.38
1089,2022-12-28,7759,24596.03
1090,2022-12-29,6457,20468.69
1091,2022-12-30,7284,23090.28
1092,2022-12-31,0,0.00


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1093 entries, 0 to 1092
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    1093 non-null   datetime64[ns]
 1   Units   1093 non-null   int64         
 2   Sales   1093 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 25.7 KB


In [51]:
X = df.drop(['Units', 'Sales'], axis=1)
y = df['Units']
X

,Date
0,2020-01-01
1,2020-01-02
2,2020-01-03
3,2020-01-04
4,2020-01-05
...,...
1088,2022-12-27
1089,2022-12-28
1090,2022-12-29
1091,2022-12-30


### Preprocess

In [ ]:

num_features = X

preprocessor = ColumnTransformer(
    [
        ('StandardScaler', StandardScaler(), num_features),
    ]
) 

In [ ]:
X = preprocessor.fit_transform()

### Evaluate models

In [ ]:
def evaluate_model(y_true, y_pred):
    rmse = root_mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return rmse, r2

NameError: name 'LinearRegression' is not defined

In [ ]:
models = {
    'LinearRegression': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'XGBRegressor': XGBRegressor()
}

model_list = []
r2_list = []
rmse_list = []
results = []
param_grid = {'': [0, 1]}


for model_name, model in models.items():
    time_series_split = TimeSeriesSplit(n_splits=6)
    cv_scores = cross_validate(model, X, y, cv=time_series_split)
    #cv_results = cross_val_score(model, X_train, y_train, cv=cv)
    
    # Fine tune hyperparams
    gs = GridSearchCV(model, cv=time_series_split, param_grid=param_grid)
    gs.fit(X, y)
    results.append(cv_scores['test_score'].mean())

    #train_rmse, train_r2 = evaluate_model() # ???
    #y_pred_test = model.predict(X_test)
    #test_rmse, test_r2 = evaluate_model(y_test, y_pred_test)
    print(model_name)
    print('Model performance for Training set')
    print(f'- RMSE: {train_rmse}')
    print(f'- R2: {train_r2}')
    print('-----------------------------')
    print('Model performance for Test set')
    print(f'- RMSE: {test_rmse}')
    print(f'- R2: {test_r2}')

plt.boxplot(results, labels=models.keys())
plt.show()

In [ ]:
import numpy as np
import xgboost as xgb
#para = param[LinearRegression()]
# or this, not sure 
param_grid = {'learning_rate': [0.01, 0.1, 0.5, 0.9],
              'n_estimators': [100,200,400]
}
time_series_split = TimeSeriesSplit(test_size=90)

#model = LinearRegression()
model = XGBRegressor()
xgb.DMatrix()
# Perform randomized grid search on train data and find best params
gs = GridSearchCV(model, param_grid, cv=time_series_split, scoring='neg_root_mean_squared_error')
gs.fit(X, y)

print("Best CV score: ", np.abs(gs.best_score_))

#model.set_params(**gs.best_params_)
#gs.best_score_

#cv_scores = cross_validate(model, X, y, cv=time_series_split)
#print("CV score: ", cv_scores['test_score'].mean())

ValueError: 
All the 60 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Vector\Documents\GitHub\Demand-Forecasting-Web-app\venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Vector\Documents\GitHub\Demand-Forecasting-Web-app\venv\Lib\site-packages\xgboost\core.py", line 726, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "c:\Users\Vector\Documents\GitHub\Demand-Forecasting-Web-app\venv\Lib\site-packages\xgboost\sklearn.py", line 1143, in fit
    train_dmatrix, evals = _wrap_evaluation_matrices(
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\